# Fonction moyenne pondérée

Cette fonction permet de pondérer les évènements en fonction de leur anciennetée.

In [131]:
import pandas as pd
import numpy as np

In [132]:
folder="C:\\Users\\quentin.mascart\\Documents\\Stage DAM\\CibNav\\"

On charge les données et on les tris par ancienneté et par l'unité a partir de laquelle on pondère. Ici les navires

In [133]:
Historique_prescriptions=pd.read_csv(folder+"Données\\Historique_Visite.csv")
Historique_prescriptions=Historique_prescriptions.sort_values(by=['ID Nav Flotteur','Date Visite'])

Création d'un vecteur de poids

In [134]:
lag=max(Historique_prescriptions['ID Nav Flotteur'].value_counts())
#On génère lag-points espacés logarithmiquement de 1 à 10
poids=list(np.logspace(0, 1, lag, endpoint=True))[::-1] #Pour inverser la liste

Cette fonction prend en entrée un dataframe, un vecteur de pondération, la variable a pondérer et l'unité sur laquelle on pondère. Le nombre de pondération disponible détermine le nombre de visites considérer dans la pondération.

In [139]:
def pondération(df,poids,variable,unite):
    nom=variable+' pondérée'
    df[nom]=0
    for i in range(0,len(poids)):
        df["poids"]=poids[i]*df.groupby([unite])[variable].shift(i+1)
        df[nom] = df.fillna(0)[nom] + df.fillna(0)["poids"]
    del (df['poids'])
    return(df)

In [138]:
Historique_prescriptions=pondération(Historique_prescriptions,poids,'Nombre Prescriptions','ID Nav Flotteur')
Historique_prescriptions=pondération(Historique_prescriptions,poids,'Nombre Prescriptions Majeurs','ID Nav Flotteur')
Historique_prescriptions

,ID Gin Visite,Date Visite,ID Nav Flotteur,Nombre Prescriptions,Nombre Prescriptions Majeurs,Nombre Prescriptions pondérée,Nombre Prescriptions Majeurs pondérée
7576,92531,2016-03-10T00:00:00.000+01:00,3774.0,1.0,0.0,0.000000,0.000000
7575,104235,2017-03-02T00:00:00.000+01:00,3774.0,3.0,0.0,10.000000,0.000000
7574,116998,2018-03-21T00:00:00.000+01:00,3774.0,13.0,1.0,36.309573,0.000000
7573,119284,2018-04-30T00:00:00.000+02:00,3774.0,0.0,0.0,152.909792,10.000000
7572,129696,2019-03-28T00:00:00.000+01:00,3774.0,3.0,0.0,96.479556,6.309573


In [ ]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [140]:
Historique=pd.read_csv(folder+"Données\\DataTrain.csv")

In [144]:
Historique_prescriptions.head()

,ID Gin Visite,Date Visite,ID Nav Flotteur,Nombre Prescriptions,Nombre Prescriptions Majeurs,Nombre Prescriptions pondérée,Nombre Prescriptions Majeurs pondérée
7576,92531,2016-03-10T00:00:00.000+01:00,3774.0,1.0,0.0,0.000000,0.000000
7575,104235,2017-03-02T00:00:00.000+01:00,3774.0,3.0,0.0,10.000000,0.000000
7574,116998,2018-03-21T00:00:00.000+01:00,3774.0,13.0,1.0,36.309573,0.000000
7573,119284,2018-04-30T00:00:00.000+02:00,3774.0,0.0,0.0,152.909792,10.000000
7572,129696,2019-03-28T00:00:00.000+01:00,3774.0,3.0,0.0,96.479556,6.309573


In [155]:
Un=Historique.sort_values(by=['ID Nav Flotteur','Date Visite']).head()
Un

,Annee Construction,Annee Visite,Date Visite,Delai Visites,ID Gin Visite,ID Nav Flotteur,Longueur Hors Tout,Nombre Prescriptions,Nombre Prescriptions Hist,Nombre Prescriptions Majeurs,Nombre Prescriptions Majeurs Hist,Prescriptions,Prescriptions Majeures,Sitrep History
0,1971.0,2016,2016-03-10,365.0,92531,3774.0,6.03,1.0,1.000000,0.0,0.000000,1,0,0
1,1971.0,2017,2017-03-02,357.0,104235,3774.0,6.03,3.0,1.000000,0.0,0.000000,1,0,0
2,1971.0,2018,2018-03-21,384.0,116998,3774.0,6.03,13.0,2.000000,1.0,0.000000,1,1,0
3,1971.0,2018,2018-04-30,40.0,119284,3774.0,6.03,0.0,5.666667,0.0,0.333333,0,0,0
4,1971.0,2019,2019-03-28,332.0,129696,3774.0,6.03,3.0,4.250000,0.0,0.250000,1,0,0


In [199]:
def decreasing_mean(presc):
    presc_list = presc.to_list()
    presc_mean= {}
    try:
        for i, val in enumerate(presc_list):
            if i > 0:
                presc_mean[i] = np.mean(presc_list[:i])
            else:
                presc_mean[i] = 0
        return pd.Series(presc_mean)
    except TypeError as e:
        return presc
    
def add_prescriptions_history(df):
    df_prescr_hist = df.copy()
    for presc in ['Nombre Prescriptions', 'Nombre Prescriptions Majeurs']:
        col_name = '{}_hist'.format(presc)
        df_prescr_hist[col_name] = df_prescr_hist.groupby('ID Nav Flotteur')[presc].transform(decreasing_mean)
    return df_prescr_hist

In [200]:
decreasing_mean(Historique['Nombre Prescriptions'])

0        0.000000
1        1.000000
2        2.000000
3        5.666667
4        4.250000
           ...   
27132    3.563689
27133    3.563557
27134    3.563426
27135    3.563331
27136    3.563200
Length: 27137, dtype: float64

In [191]:
Historique.head()

,Annee Construction,Annee Visite,Date Visite,Delai Visites,ID Gin Visite,ID Nav Flotteur,Longueur Hors Tout,Nombre Prescriptions,Nombre Prescriptions Hist,Nombre Prescriptions Majeurs,Nombre Prescriptions Majeurs Hist,Prescriptions,Prescriptions Majeures,Sitrep History
0,1971.0,2016,2016-03-10,365.0,92531,3774.0,6.03,1.0,1.000000,0.0,0.000000,1,0,0
1,1971.0,2017,2017-03-02,357.0,104235,3774.0,6.03,3.0,1.000000,0.0,0.000000,1,0,0
2,1971.0,2018,2018-03-21,384.0,116998,3774.0,6.03,13.0,2.000000,1.0,0.000000,1,1,0
3,1971.0,2018,2018-04-30,40.0,119284,3774.0,6.03,0.0,5.666667,0.0,0.333333,0,0,0
4,1971.0,2019,2019-03-28,332.0,129696,3774.0,6.03,3.0,4.250000,0.0,0.250000,1,0,0


In [201]:
A=add_prescriptions_history(Historique)

In [202]:
A.head()

,Annee Construction,Annee Visite,Date Visite,Delai Visites,ID Gin Visite,ID Nav Flotteur,Longueur Hors Tout,Nombre Prescriptions,Nombre Prescriptions Hist,Nombre Prescriptions Majeurs,Nombre Prescriptions Majeurs Hist,Prescriptions,Prescriptions Majeures,Sitrep History,Nombre Prescriptions_hist,Nombre Prescriptions Majeurs_hist
0,1971.0,2016,2016-03-10,365.0,92531,3774.0,6.03,1.0,1.000000,0.0,0.000000,1,0,0,0.000000,0.000000
1,1971.0,2017,2017-03-02,357.0,104235,3774.0,6.03,3.0,1.000000,0.0,0.000000,1,0,0,1.000000,0.000000
2,1971.0,2018,2018-03-21,384.0,116998,3774.0,6.03,13.0,2.000000,1.0,0.000000,1,1,0,2.000000,0.000000
3,1971.0,2018,2018-04-30,40.0,119284,3774.0,6.03,0.0,5.666667,0.0,0.333333,0,0,0,5.666667,0.333333
4,1971.0,2019,2019-03-28,332.0,129696,3774.0,6.03,3.0,4.250000,0.0,0.250000,1,0,0,4.250000,0.250000


In [183]:
print(L[:0])
np.mean(L[:2])

[]


2.0